In [247]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [248]:
import xml.etree.ElementTree as etree

In [271]:
def getContext(root):
    def get_params(eqn, all_variables):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        res = filter(lambda x : x in all_variables, res)
        return res
    #
    inputs = root.find('inputs')
    outputs = root.find('outputs')
    internal_connections = root.find('internal_connections')
    external_input_connections = root.find('external_input_connections')
    external_output_connections = root.find('external_output_connections')
    components = root.find('components')
    atomics = components.findall('atomicRef')
    all_variables = []
    for atomic in atomics:
        all_variables.append(atomic.get('name'))
    
    coupled = components.findall('coupled')
    
    # TODO : coupled_
    coupled_ = []

    #
    inputs_ = []
    outputs_ = []
    internal_connections_ = []
    external_input_connections_ = []
    external_output_connections_ = []
    atomics_ = []
    # Atomics
    for atomic in atomics:
        name  = atomic.get('name')
        model = atomic.get('model')
        ports = atomic.findall('port')
        parameters = atomic.findall('parameter')
        
        # TODO : MEJORAR ESTO
        class_ = name
        if model == 'QSS1':
            class_ = 'QSS1'
        if model == 'Cte':
            class_ = 'Cte'
        #
        
        atomics_.append({
            'name' : name,
            'class' : class_,
            'model' : model,
            'in_ports' : [{'name' : port.get('name'), 'type' : port.get('type')} 
                          for port in ports 
                              if port.get('type') == 'in' or
                                 port.get('type') =='in_plus' or
                                 port.get('type') == 'in_minus'],
            'out_ports' :  [{'name' : port.get('name'), 'type' : port.get('type')} 
                          for port in ports if port.get('type') == 'out'],
            'parameters' : [{'name' : param.get('name'), 
                             'function' : param.get('function'),
                             'function_params' : get_params(param.get('function'), all_variables)} 
                            for param in parameters]
        })
    # Internal Connections
    for connection in internal_connections:
        internal_connections_.append({
            'component_from' : connection.get('component_from'),
            'component_to' : connection.get('component_to'),
            'port_from' : connection.get('port_from'),
            'port_to' : connection.get('port_to')
        })
    # Input ports
    for in_port in inputs:
        inputs_.append(in_port.get('name'))
    # Output ports
    for out_port in outputs:
        outputs_.append(out_port.get('name'))
    context = {
        'inputs' : inputs_,
        'outputs' : outputs_,
        'atomics' : atomics_,
        'internal_connections' : internal_connections_,
        'external_input_connections' : external_input_connections_,
        'external_output_connections' : external_output_connections_
    }
    return context

In [272]:
def generate_h_cpp(context, srcFolder):
    #
    def save(src, template, ext, context):
        srcFolder = src + context['name'] + ext
        with open(srcFolder, 'wb') as f:
            f.write(render_template(template + ext, context))
    #
    auxs = filter(lambda x : x['model'] == 'Aux', context['atomics'])
    fpms = filter(lambda x : x['model'] == 'Fpm', context['atomics'])
    fts  = filter(lambda x : x['model'] == 'Ftot', context['atomics'])
    for aux in auxs:
        parameters = aux['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        aux['parameters'] = parameters_
    for fpm in fpms:
        parameters = fpm['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        fpm['parameters'] = parameters_
    for ft in fts:
        in_ports_plus, in_ports_minus = [], []
        parameters = ft['parameters']
        parameters_ = []
        for p in parameters:
            for x in p['function_params']:
                p['function'] = p['function'].replace(x, 'val_' + x)
            parameters_.append(p)
        in_ports_plus = filter(lambda x : x['type'] == 'in_plus', ft['in_ports'])
        in_ports_minus = filter(lambda x : x['type'] == 'in_minus', ft['in_ports'])
        ft['parameters'] = parameters_
        ft['in_ports_plus'] = in_ports_plus
        ft['in_ports_minus'] = in_ports_minus
    # Fpms
    for fpm in fpms:
        template = 'template-Aux-Fpm'
        for ext in ['.h', '.cpp']:
            save(srcFolder, template, ext, fpm)
    
    # Auxs
    for aux in auxs:
        template = 'template-Aux-Fpm'
        for ext in ['.h', '.cpp']:
            save(srcFolder, template, ext, aux)
    # Ftot's
    for ft in fts:
        template = 'template-Ftot'
        for ext in ['.h', '.cpp']:
            save(srcFolder, template, ext, ft)
    
    # Ev-file
    ev = []
    for aux in auxs:
        assert len(aux['parameters']) == 1
        param = aux['parameters'][0]
        if param['function_params'] == []:
            ev.append({'name' : aux['name'], 'value' : param['function']})
    template = 'template-ev.ev'
    with open(srcFolder + 'events.ev', 'wb') as f:
        f.write(render_template(template, {'variables' : ev}))
            
    # Reg-file
    template = 'template-reg.cpp'
    with open(srcFolder + 'reg.cpp', 'wb') as f:
        context = {'atomics' : [atomic['name'] for atomic in auxs + fpms + fts]}
        f.write(render_template(template, context))
    return True

def generate_ma(context, dst_file):
    template = 'template-ma.ma'
    with open(dst_file, 'wb') as f:
        f.write(render_template(template, context))
    return True

In [273]:
DIR_DEVSML = 'test.xml'
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_DEVSML, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

context = getContext(root)
generate_h_cpp(context, 'goodwin/src/')
generate_ma(context, 'goodwin/goodwin-traduccion.ma')

True

In [274]:
print context

{'inputs': ['constantEmploymentRate', 'velocityOfMoney', 'rateOfAppreciation'], 'outputs': ['LaborProductivity', 'wageRate', 'Population', 'Capital'], 'atomics': [{'in_ports_minus': [], 'name': 'LaborProductivityTot', 'parameters': [], 'out_ports': [{'type': 'out', 'name': 'out'}], 'in_ports': [{'type': 'in_plus', 'name': 'chgLaborProductivity'}], 'model': 'Ftot', 'in_ports_plus': [{'type': 'in_plus', 'name': 'chgLaborProductivity'}], 'class': 'LaborProductivityTot'}, {'name': 'LaborProductivity', 'parameters': [{'function': '1', 'name': 'x0', 'function_params': []}, {'function': '0.0001', 'name': 'dQRel', 'function_params': []}, {'function': '0.0001', 'name': 'dQMin', 'function_params': []}], 'out_ports': [{'type': 'out', 'name': 'out'}], 'in_ports': [{'type': 'in', 'name': 'in'}], 'model': 'QSS1', 'class': 'QSS1'}, {'in_ports_minus': [], 'name': 'wageRateTot', 'parameters': [], 'out_ports': [{'type': 'out', 'name': 'out'}], 'in_ports': [{'type': 'in_plus', 'name': 'chgWageRate'}], 'm